## holdout predictions

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import t
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
import pickle
% matplotlib inline
plt.style.use('seaborn')
pd.set_option('display.max_columns', 300)

In [2]:

holdout = pd.read_csv('holdout_data.csv')

In [3]:
holdout.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23
0,5501,180000,2,2,1,44,0,0,0,0,0,0,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000
1,28857,130000,2,2,1,48,-2,-2,-2,-2,-2,-2,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849
2,11272,60000,2,1,1,43,-1,3,2,0,0,-1,495,330,495,330,165,340,0,330,0,0,340,0
3,8206,240000,1,1,1,42,0,0,0,0,0,0,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893
4,6362,100000,2,2,1,28,2,0,0,0,0,2,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0


In [4]:
del holdout['Unnamed: 0']


In [5]:
rename_list = ["max_credit", "gender", "education", "marital_status", "age",
               "pay_status_sep", "pay_status_aug", "pay_status_jul", "pay_status_jun", "pay_status_may", "pay_status_apr",
               "bill_sep", "bill_aug", "bill_jul", "bill_jun", "bill_may", "bill_apr",
               "payment_sep", "payment_aug", "payments_jul", "payment_jun", "payment_may", "payment_apr",
                "default"]
col_rename = dict(zip(holdout.columns,rename_list))
holdout = holdout.rename(columns=col_rename)

In [6]:
holdout.head()

,max_credit,gender,education,marital_status,age,pay_status_sep,pay_status_aug,pay_status_jul,pay_status_jun,pay_status_may,pay_status_apr,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr
0,180000,2,2,1,44,0,0,0,0,0,0,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000
1,130000,2,2,1,48,-2,-2,-2,-2,-2,-2,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849
2,60000,2,1,1,43,-1,3,2,0,0,-1,495,330,495,330,165,340,0,330,0,0,340,0
3,240000,1,1,1,42,0,0,0,0,0,0,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893
4,100000,2,2,1,28,2,0,0,0,0,2,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0


In [7]:
holdout.pay_status_apr.value_counts()

 0    4053
-1    1456
-2    1232
 2     688
 3      44
 4      11
 7       8
 6       5
 5       4
Name: pay_status_apr, dtype: int64

In [8]:
holdout.pay_status_aug.value_counts()

 0    3926
-1    1524
-2     969
 2     960
 3      75
 4      29
 5       6
 7       4
 6       4
 1       4
Name: pay_status_aug, dtype: int64

In [9]:
holdout.shape

(7501, 23)

In [10]:
holdout.isna().sum()

max_credit        0
gender            0
education         0
marital_status    0
age               0
pay_status_sep    0
pay_status_aug    0
pay_status_jul    0
pay_status_jun    0
pay_status_may    0
pay_status_apr    0
bill_sep          0
bill_aug          0
bill_jul          0
bill_jun          0
bill_may          0
bill_apr          0
payment_sep       0
payment_aug       0
payments_jul      0
payment_jun       0
payment_may       0
payment_apr       0
dtype: int64

In [11]:
holdout.marital_status.value_counts()

2    3938
1    3464
3      89
0      10
Name: marital_status, dtype: int64

In [12]:
holdout.marital_status = np.where(holdout.marital_status < 1, 3, holdout.marital_status)

In [13]:
holdout['young'] = np.where(holdout.age < 30, 1,0)
holdout['avg_bill'] = ((holdout.bill_sep + holdout.bill_aug + holdout.bill_jul + holdout.bill_jun + holdout.bill_may + holdout.bill_apr)/6)
holdout['avg_payment'] = ((holdout.payment_sep + holdout.payment_aug + holdout.payments_jul + holdout.payment_jun + holdout.payment_may + holdout.payment_apr)/6)




In [14]:
gender_dummies = pd.get_dummies(holdout.gender, prefix='gender', drop_first=True)
education_dummies = pd.get_dummies(holdout.education, prefix='education', drop_first=True)
married_dummies = pd.get_dummies(holdout.marital_status, prefix='marital_status', drop_first=True)
pay_sep_dummies = pd.get_dummies(holdout.pay_status_sep, prefix='pay_status_sep', drop_first=True)
pay_aug_dummies = pd.get_dummies(holdout.pay_status_aug, prefix='pay_status_aug', drop_first=True)
pay_jul_dummies = pd.get_dummies(holdout.pay_status_jul, prefix='pay_status_jul', drop_first=True)
pay_jun_dummies = pd.get_dummies(holdout.pay_status_jun, prefix='pay_status_jun', drop_first=True)
pay_may_dummies = pd.get_dummies(holdout.pay_status_may, prefix='pay_status_may', drop_first=True)
pay_apr_dummies = pd.get_dummies(holdout.pay_status_apr, prefix='pay_status_apr', drop_first=True)
young_dummies = pd.get_dummies(holdout.young, prefix='young', drop_first=True)

dummie_data = pd.concat([gender_dummies,education_dummies,
                         married_dummies,
                         pay_sep_dummies,pay_aug_dummies,
                        pay_jul_dummies,pay_jun_dummies,
                        pay_may_dummies,pay_apr_dummies,
                        young_dummies],axis=1)
dummie_data.head(1)

,gender_2,education_1,education_2,education_3,education_4,education_5,education_6,marital_status_2,marital_status_3,pay_status_sep_-1,pay_status_sep_0,pay_status_sep_1,pay_status_sep_2,pay_status_sep_3,pay_status_sep_4,pay_status_sep_5,pay_status_sep_6,pay_status_sep_7,pay_status_sep_8,pay_status_aug_-1,pay_status_aug_0,pay_status_aug_1,pay_status_aug_2,pay_status_aug_3,pay_status_aug_4,pay_status_aug_5,pay_status_aug_6,pay_status_aug_7,pay_status_jul_-1,pay_status_jul_0,pay_status_jul_2,pay_status_jul_3,pay_status_jul_4,pay_status_jul_5,pay_status_jul_6,pay_status_jul_7,pay_status_jul_8,pay_status_jun_-1,pay_status_jun_0,pay_status_jun_2,pay_status_jun_3,pay_status_jun_4,pay_status_jun_5,pay_status_jun_7,pay_status_may_-1,pay_status_may_0,pay_status_may_2,pay_status_may_3,pay_status_may_4,pay_status_may_5,pay_status_may_6,pay_status_may_7,pay_status_apr_-1,pay_status_apr_0,pay_status_apr_2,pay_status_apr_3,pay_status_apr_4,pay_status_apr_5,pay_status_apr_6,pay_status_apr_7,young_1
0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [15]:
dummie_data.shape

(7501, 61)

In [16]:
holdout_list = list(dummie_data.columns.values)
holdout_list

['gender_2',
 'education_1',
 'education_2',
 'education_3',
 'education_4',
 'education_5',
 'education_6',
 'marital_status_2',
 'marital_status_3',
 'pay_status_sep_-1',
 'pay_status_sep_0',
 'pay_status_sep_1',
 'pay_status_sep_2',
 'pay_status_sep_3',
 'pay_status_sep_4',
 'pay_status_sep_5',
 'pay_status_sep_6',
 'pay_status_sep_7',
 'pay_status_sep_8',
 'pay_status_aug_-1',
 'pay_status_aug_0',
 'pay_status_aug_1',
 'pay_status_aug_2',
 'pay_status_aug_3',
 'pay_status_aug_4',
 'pay_status_aug_5',
 'pay_status_aug_6',
 'pay_status_aug_7',
 'pay_status_jul_-1',
 'pay_status_jul_0',
 'pay_status_jul_2',
 'pay_status_jul_3',
 'pay_status_jul_4',
 'pay_status_jul_5',
 'pay_status_jul_6',
 'pay_status_jul_7',
 'pay_status_jul_8',
 'pay_status_jun_-1',
 'pay_status_jun_0',
 'pay_status_jun_2',
 'pay_status_jun_3',
 'pay_status_jun_4',
 'pay_status_jun_5',
 'pay_status_jun_7',
 'pay_status_may_-1',
 'pay_status_may_0',
 'pay_status_may_2',
 'pay_status_may_3',
 'pay_status_may_4',
 'pa

In [17]:
holdout_1 = holdout.drop(['gender','education','marital_status','pay_status_sep',
               'pay_status_aug','pay_status_jul','pay_status_jun',
               'pay_status_may','pay_status_apr','young'],axis=1)

In [18]:
holdout_1.head()

,max_credit,age,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr,avg_bill,avg_payment
0,180000,44,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000,167239.666667,8000.000000
1,130000,48,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849,865.833333,1007.333333
2,60000,43,495,330,495,330,165,340,0,330,0,0,340,0,359.166667,111.666667
3,240000,42,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893,59507.666667,6991.500000
4,100000,28,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0,66593.333333,2352.166667


In [19]:
transformed_holdout = pd.concat([holdout_1,dummie_data], axis=1)

In [20]:
transformed_holdout.head()

,max_credit,age,bill_sep,bill_aug,bill_jul,bill_jun,bill_may,bill_apr,payment_sep,payment_aug,payments_jul,payment_jun,payment_may,payment_apr,avg_bill,avg_payment,gender_2,education_1,education_2,education_3,education_4,education_5,education_6,marital_status_2,marital_status_3,pay_status_sep_-1,pay_status_sep_0,pay_status_sep_1,pay_status_sep_2,pay_status_sep_3,pay_status_sep_4,pay_status_sep_5,pay_status_sep_6,pay_status_sep_7,pay_status_sep_8,pay_status_aug_-1,pay_status_aug_0,pay_status_aug_1,pay_status_aug_2,pay_status_aug_3,pay_status_aug_4,pay_status_aug_5,pay_status_aug_6,pay_status_aug_7,pay_status_jul_-1,pay_status_jul_0,pay_status_jul_2,pay_status_jul_3,pay_status_jul_4,pay_status_jul_5,pay_status_jul_6,pay_status_jul_7,pay_status_jul_8,pay_status_jun_-1,pay_status_jun_0,pay_status_jun_2,pay_status_jun_3,pay_status_jun_4,pay_status_jun_5,pay_status_jun_7,pay_status_may_-1,pay_status_may_0,pay_status_may_2,pay_status_may_3,pay_status_may_4,pay_status_may_5,pay_status_may_6,pay_status_may_7,pay_status_apr_-1,pay_status_apr_0,pay_status_apr_2,pay_status_apr_3,pay_status_apr_4,pay_status_apr_5,pay_status_apr_6,pay_status_apr_7,young_1
0,180000,44,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000,167239.666667,8000.000000,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,130000,48,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849,865.833333,1007.333333,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,60000,43,495,330,495,330,165,340,0,330,0,0,340,0,359.166667,111.666667,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,240000,42,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893,59507.666667,6991.500000,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,100000,28,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0,66593.333333,2352.166667,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [21]:
transformed_holdout.shape

(7501, 77)

In [22]:
with open('model.pickle','rb') as f:
    model = pickle.load(f)

In [23]:
with open('scaler.pickle','rb') as f:
    scaler = pickle.load(f)

In [26]:
transformed_holdout['pay_status_aug_8'] = 0

In [28]:
transformed_holdout['pay_status_jun_8'] = 0
transformed_holdout['pay_status_may_8'] = 0
transformed_holdout['pay_status_apr_8'] = 0
transformed_holdout['pay_status_jul_1'] = 0
transformed_holdout['pay_status_jun_1'] = 0

In [34]:
transformed_holdout['pay_status_jun_6'] = 0

In [35]:
transformed_holdout.shape

(7501, 84)

In [44]:
transformed_holdout = scaler.transform(transformed_holdout)

/Users/heymanjake/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [45]:
bake_off_pred = model.predict(transformed_holdout)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/heymanjake/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    1.7s remaining:    5.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.7s finished


In [46]:
bake_off_pred

array([0, 1, 1, ..., 1, 1, 1])

In [48]:
pd.DataFrame(bake_off_pred).to_csv('bake_off_preds_Jacob_Heyman')

In [42]:
set(bake_off_pred)

{0, 1}

In [47]:
sum(bake_off_pred)

3476